In [5]:
import sys
import os

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

from run import main

# Finds thresholds for IH-confidence and IH-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/bioresponse",
    train_name="bioresponse_train",
    measure="ih",  # or "if"
    metric="macro-f1",
    steps=[#"build", 
           #"results", "reports", "plot", 
        "grid", "heuristic", #"brute", "annealing" 
          ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 0.825
Rejecting threshold(s): 0.86
Associated minimum cost: 0.913519

Uncertainty-based rejection
Filtering threshold(s): 0.6125
Rejecting threshold(s): 0.54
Associated minimum cost: 1.065509


In [6]:
# Finds thresholds for IF-confidence and IF-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/bioresponse",
    train_name="bioresponse_train",
    measure="if",  # or "if"
    metric="macro-f1",
    steps=[#"build",
           #"results", "reports", "plot", 
           "grid", "heuristic", #"brute", "annealing"
    ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 1.0
Rejecting threshold(s): 0.78
Associated minimum cost: 0.961116

Uncertainty-based rejection
Filtering threshold(s): 0.975
Rejecting threshold(s): 0.52
Associated minimum cost: 1.105974


In [1]:
from thresholds_tester import ThresholdsTester

import sys
import os

# Test the thresholds found in the test set

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

tester = ThresholdsTester(
    data_path="(...)/RefineAI/experiments/bioresponse",
    train_name="bioresponse_train",
    test_name="bioresponse_test",
    metric="macro-f1",
    ih_uncertainty=[0.6125, 0.54], #filtering, rejecting
    ih_confidence=[0.8250, 0.86],
    if_uncertainty=[0.975, 0.52],
    if_confidence=[1, 0.78]
)

tester.final_measures() 
final, macro = tester.final_results()

In [2]:
# Reorganizes the results 

final = final.reindex([2, 1, 0, 5, 4, 3, 8, 7, 6, 11, 10, 9, 12, 13]) 
final

,measure,metric,T_value,thresholds,precision_class_0,precision_class_1,recall_class_0,recall_class_1,rate_accep_0,rate_accep_1,mean_conf_uncert,f1_0,f1_1,f1-macro,acp_a
2,ih,C,0.000,0.86,0.923,0.905,0.840,0.956,0.413,0.553,0.883,0.879546,0.929801,0.904674,0.4830
1,ih,C,0.175,0.86,0.799,0.809,0.767,0.837,0.915,0.912,0.963,0.782673,0.822762,0.802717,0.9135
0,ih,C,0.175,0.00,0.781,0.795,0.748,0.823,1.000,1.000,0.941,0.764144,0.808758,0.786451,1.0000
5,ih,U,0.000,0.54,0.822,0.823,0.770,0.866,0.828,0.864,0.748,0.795151,0.843953,0.819552,0.8460
4,ih,U,0.387,0.54,0.782,0.776,0.726,0.824,0.948,0.920,0.867,0.752960,0.799280,0.776120,0.9340
3,ih,U,0.387,0.00,0.749,0.773,0.723,0.795,1.000,1.000,0.843,0.735770,0.783846,0.759808,1.0000
8,if,C,0.000,0.78,0.865,0.867,0.814,0.905,0.700,0.791,0.864,0.838725,0.885593,0.862159,0.7455
7,if,C,0.000,0.78,0.865,0.867,0.814,0.905,0.700,0.791,0.864,0.838725,0.885593,0.862159,0.7455
6,if,C,0.000,0.00,0.802,0.793,0.738,0.846,1.000,1.000,0.814,0.768670,0.818643,0.793657,1.0000
11,if,U,0.000,0.52,0.816,0.808,0.754,0.859,0.891,0.908,0.735,0.783776,0.832720,0.808248,0.8995


In [3]:
# Calculates the cost for each solution, used to statistical comparison

macro = macro[['measure', 'metric', 'f1-macro','acp_a','mean_conf_uncert']]
macro = round(macro,3)

macro['cost'] = 4*(1-macro['f1-macro']) + (1-(macro['acp_a'])) + (1-macro['mean_conf_uncert'])

macro_confidence = macro.reindex([1, 7, 12])
macro_uncertainty = macro.reindex([4, 10, 13])

print(','.join([f"'{x:.3}'" for x in macro_uncertainty.iloc[:,-1]]))

'1.09','1.11','1.11'


In [4]:
print(','.join([f"'{x:.3}'" for x in macro_confidence.iloc[:,-1]]))

'0.911','0.942','1.01'
